# Analyse des Winrates - League of Legends
## Comparaison entre Patches

Ce notebook permet d'analyser les variations de winrate des champions entre deux patches.

---

## 📦 Installation des dépendances

In [1]:
!pip install requests beautifulsoup4 pandas lxml openpyxl

## 📚 Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import Dict, List, Optional
import time
from datetime import datetime

print("✓ Imports réussis!")

✓ Imports réussis!


## ⚙️ Configuration

**MODIFIE ICI LES PATCHES À COMPARER**

In [3]:
# URLs et headers
BASE_URL = "https://lolalytics.com/lol"
LANES = ["top", "jungle", "middle", "bottom", "support"]

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
}

# ========== CONFIGURATION DU SCRAPING ==========
TIER = "diamond_plus"  # master_plus, diamond_plus, platinum_plus, etc.
REGION = "all"  # all, euw, na, kr, etc.
PATCH_1 = "15.24"  # Premier patch (ancien)
PATCH_2 = "16.1"   # Second patch (nouveau)
# ================================================

print(f"Configuration:")
print(f"  Tier: {TIER}")
print(f"  Region: {REGION}")
print(f"  Patches à comparer: {PATCH_1} vs {PATCH_2}")

Configuration:
  Tier: diamond_plus
  Region: all
  Patches à comparer: 15.24 vs 16.1


## 🎮 Liste des champions

In [4]:
CHAMPIONS = [
    "aatrox", "ahri", "akali", "akshan", "alistar", "ambessa", "amumu", "anivia", "annie", "aphelios",
    "mel", "ashe", "aurelionsol", "aurora", "azir", "bard", "belveth", "blitzcrank", "brand", "braum", "briar",
    "caitlyn", "camille", "cassiopeia", "chogath", "corki", "darius", "diana", "draven", "drmundo",
    "ekko", "elise", "evelynn", "ezreal", "fiddlesticks", "fiora", "fizz", "galio", "gangplank",
    "garen", "gnar", "gragas", "graves", "gwen", "hecarim", "heimerdinger", "hwei", "illaoi", "irelia",
    "ivern", "janna", "jarvaniv", "jax", "jayce", "jhin", "jinx", "kaisa", "kalista", "karma",
    "karthus", "kassadin", "katarina", "kayle", "kayn", "kennen", "khazix", "kindred", "kled",
    "kogmaw", "ksante", "leblanc", "leesin", "leona", "lillia", "lissandra", "lucian", "lulu", "lux",
    "malphite", "malzahar", "maokai", "masteryi", "milio", "missfortune", "mordekaiser", "morgana",
    "naafiri", "nami", "nasus", "nautilus", "neeko", "nidalee", "nilah", "nocturne", "nunu", "olaf",
    "orianna", "ornn", "pantheon", "poppy", "pyke", "qiyana", "quinn", "rakan", "rammus", "reksai",
    "rell", "renata", "renekton", "rengar", "riven", "rumble", "ryze", "samira", "sejuani", "senna",
    "seraphine", "sett", "shaco", "shen", "shyvana", "singed", "sion", "sivir", "skarner", "smolder",
    "sona", "soraka", "swain", "sylas", "syndra", "tahmkench", "taliyah", "talon", "taric", "teemo",
    "thresh", "tristana", "trundle", "tryndamere", "twistedfate", "twitch", "udyr", "urgot", "varus",
    "vayne", "veigar", "velkoz", "vex", "vi", "viego", "viktor", "vladimir", "volibear", "warwick",
    "wukong", "xayah", "xerath", "xinzhao", "yasuo", "yone", "yorick", "yuumi", "zac", "zed", "zeri",
    "ziggs", "zilean", "zoe", "zyra"
]

print(f"✓ {len(CHAMPIONS)} champions configurés")

✓ 170 champions configurés


## 🔧 Fonctions de scraping

In [5]:
def get_html(url: str, params: Optional[Dict] = None) -> Optional[str]:
    """Récupère le HTML d'une page"""
    try:
        response = requests.get(url, headers=HEADERS, params=params, timeout=30)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException:
        return None


def extract_qwik_json(html: str) -> Optional[Dict]:
    """Extrait les données JSON de la page"""
    soup = BeautifulSoup(html, 'lxml')
    qwik_script = soup.find('script', type='qwik/json')
    if not qwik_script or not qwik_script.string:
        return None
    try:
        return json.loads(qwik_script.string)
    except json.JSONDecodeError:
        return None


def decode_ref(ref_str: str, objs: List) -> any:
    """Décode les références dans le JSON"""
    if isinstance(ref_str, str):
        try:
            idx = int(ref_str, 36)
            if 0 <= idx < len(objs):
                return objs[idx]
        except ValueError:
            pass
    return ref_str


def find_champion_stats(objs: List) -> Optional[Dict]:
    """Trouve les statistiques du champion dans les données JSON"""
    for obj in objs:
        if isinstance(obj, dict):
            required_keys = {'wr', 'pr', 'br', 'n', 'avgWr'}
            if required_keys.issubset(set(obj.keys())):
                stats = {}
                for key in ['wr', 'avgWr', 'avgWrDelta', 'pr', 'br', 'n', 'tier']:
                    if key in obj:
                        stats[key] = decode_ref(obj[key], objs)
                return stats
    return None


def get_champion_stats(champion: str, lane: str, tier: str, region: str, patch: str) -> Optional[Dict]:
    """Récupère les statistiques d'un champion pour une lane et un patch donnés"""
    url = f"{BASE_URL}/{champion}/build/"
    params = {"tier": tier, "region": region, "patch": patch, "lane": lane}

    html = get_html(url, params)
    if not html:
        return None

    qwik_data = extract_qwik_json(html)
    if not qwik_data or 'objs' not in qwik_data:
        return None

    champion_stats = find_champion_stats(qwik_data['objs'])
    if champion_stats:
        return {
            'champion': champion,
            'lane': lane,
            'patch': patch,
            'winrate': champion_stats.get('wr'),
            'pickrate': champion_stats.get('pr'),
            'banrate': champion_stats.get('br'),
            'games': champion_stats.get('n'),
            'tier': champion_stats.get('tier')
        }
    return None


def scrape_all_champions_patch(patch: str, tier: str, region: str) -> pd.DataFrame:
    """Scrape tous les champions pour un patch donné"""
    all_data = []
    total = len(CHAMPIONS) * len(LANES)
    current = 0

    print(f"Scraping patch {patch}...")
    
    for champion in CHAMPIONS:
        for lane in LANES:
            current += 1
            stats = get_champion_stats(champion, lane, tier, region, patch)
            
            if stats:
                all_data.append(stats)
                if current % 50 == 0:
                    print(f"  [{current}/{total}] En cours...")
            
            time.sleep(0.15)
    
    df = pd.DataFrame(all_data)
    print(f"✓ Patch {patch}: {len(df)} lignes récupérées")
    return df

print("✓ Fonctions de scraping définies")

✓ Fonctions de scraping définies


## 📥 Scraping des données

In [6]:
# Timestamp de fetch
fetch_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"📅 Début du scraping: {fetch_timestamp}")
print("=" * 70)

# Scraping Patch 1
print(f"\nSCRAPING PATCH {PATCH_1}")
print("=" * 70)
df_patch_1 = scrape_all_champions_patch(PATCH_1, TIER, REGION)
df_patch_1['fetch_timestamp'] = fetch_timestamp

print(f"\n📊 Aperçu Patch {PATCH_1}:")
display(df_patch_1.head(10))

# Scraping Patch 2
print(f"\n\nSCRAPING PATCH {PATCH_2}")
print("=" * 70)
df_patch_2 = scrape_all_champions_patch(PATCH_2, TIER, REGION)
df_patch_2['fetch_timestamp'] = fetch_timestamp

print(f"\n📊 Aperçu Patch {PATCH_2}:")
display(df_patch_2.head(10))

📅 Début du scraping: 2026-01-11 11:46:37

SCRAPING PATCH 15.24
Scraping patch 15.24...
  [50/850] En cours...
  [100/850] En cours...
  [150/850] En cours...
  [200/850] En cours...
  [250/850] En cours...
  [300/850] En cours...
  [350/850] En cours...
  [400/850] En cours...
  [450/850] En cours...
  [500/850] En cours...
  [550/850] En cours...
  [600/850] En cours...
  [650/850] En cours...
  [700/850] En cours...
  [750/850] En cours...
  [800/850] En cours...
  [850/850] En cours...
✓ Patch 15.24: 847 lignes récupérées

📊 Aperçu Patch 15.24:


,champion,lane,patch,winrate,pickrate,banrate,games,tier,fetch_timestamp
0,aatrox,top,15.24,49.52,7.75,15.74,109051,B-,2026-01-11 11:46:37
1,aatrox,jungle,15.24,48.89,6.50,15.74,90049,C+,2026-01-11 11:46:37
2,aatrox,middle,15.24,48.03,0.21,15.74,2973,?,2026-01-11 11:46:37
3,aatrox,bottom,15.24,35.85,0.01,15.74,106,?,2026-01-11 11:46:37
4,aatrox,support,15.24,39.35,0.05,15.74,681,?,2026-01-11 11:46:37
5,ahri,top,15.24,53.17,0.18,1.83,2494,?,2026-01-11 11:46:37
6,ahri,jungle,15.24,22.22,0.00,1.83,18,?,2026-01-11 11:46:37
7,ahri,middle,15.24,51.54,8.70,1.83,122096,S-,2026-01-11 11:46:37
8,ahri,bottom,15.24,50.23,0.06,1.83,886,?,2026-01-11 11:46:37
9,ahri,support,15.24,47.71,0.10,1.83,1421,?,2026-01-11 11:46:37




SCRAPING PATCH 16.1
Scraping patch 16.1...
  [50/850] En cours...
  [100/850] En cours...
  [150/850] En cours...
  [200/850] En cours...
  [250/850] En cours...
  [300/850] En cours...
  [350/850] En cours...
  [400/850] En cours...
  [450/850] En cours...
  [500/850] En cours...
  [550/850] En cours...
  [600/850] En cours...
  [650/850] En cours...
  [700/850] En cours...
  [750/850] En cours...
  [800/850] En cours...
  [850/850] En cours...
✓ Patch 16.1: 783 lignes récupérées

📊 Aperçu Patch 16.1:


,champion,lane,patch,winrate,pickrate,banrate,games,tier,fetch_timestamp
0,aatrox,top,16.1,47.76,8.16,9.36,20341,C+,2026-01-11 11:46:37
1,aatrox,jungle,16.1,46.21,1.35,9.36,3315,C-,2026-01-11 11:46:37
2,aatrox,middle,16.1,51.02,0.16,9.36,394,?,2026-01-11 11:46:37
3,aatrox,support,16.1,37.18,0.03,9.36,78,?,2026-01-11 11:46:37
4,ahri,top,16.1,55.65,0.13,1.57,336,?,2026-01-11 11:46:37
5,ahri,jungle,16.1,33.33,0.00,1.57,3,?,2026-01-11 11:46:37
6,ahri,middle,16.1,50.15,8.02,1.57,19939,A,2026-01-11 11:46:37
7,ahri,bottom,16.1,40.96,0.03,1.57,83,?,2026-01-11 11:46:37
8,ahri,support,16.1,39.38,0.07,1.57,160,?,2026-01-11 11:46:37
9,akali,top,16.1,50.05,2.50,20.47,6230,B+,2026-01-11 11:46:37


## 🔀 Création du DataFrame de comparaison

Structure finale:
```
champion, lane, 
patch_X, winrate_X, pickrate_X, banrate_X, games_X, tier_X,
patch_Y, winrate_Y, pickrate_Y, banrate_Y, games_Y, tier_Y,
wr_change, pr_change, br_change
```

In [8]:
# Renommer les colonnes pour avoir la structure demandée
df_p1 = df_patch_1.copy()
df_p2 = df_patch_2.copy()

# Sauvegarder le timestamp avant de le supprimer temporairement
timestamp_value = df_p1['fetch_timestamp'].iloc[0]

# Supprimer fetch_timestamp temporairement pour le merge
df_p1 = df_p1.drop('fetch_timestamp', axis=1)
df_p2 = df_p2.drop('fetch_timestamp', axis=1)

# Renommer avec les noms de patches dynamiques
df_p1 = df_p1.rename(columns={
    'patch': f'patch_{PATCH_1}',
    'winrate': f'winrate_{PATCH_1}',
    'pickrate': f'pickrate_{PATCH_1}',
    'banrate': f'banrate_{PATCH_1}',
    'games': f'games_{PATCH_1}',
    'tier': f'tier_{PATCH_1}'
})

df_p2 = df_p2.rename(columns={
    'patch': f'patch_{PATCH_2}',
    'winrate': f'winrate_{PATCH_2}',
    'pickrate': f'pickrate_{PATCH_2}',
    'banrate': f'banrate_{PATCH_2}',
    'games': f'games_{PATCH_2}',
    'tier': f'tier_{PATCH_2}'
})

# Merge sur champion et lane
comparison = df_p1.merge(df_p2, on=['champion', 'lane'])

# Calculer les changements
comparison['wr_change'] = comparison[f'winrate_{PATCH_2}'] - comparison[f'winrate_{PATCH_1}']
comparison['pr_change'] = comparison[f'pickrate_{PATCH_2}'] - comparison[f'pickrate_{PATCH_1}']
comparison['br_change'] = comparison[f'banrate_{PATCH_2}'] - comparison[f'banrate_{PATCH_1}']

# Ajouter le timestamp
comparison['fetch_timestamp'] = timestamp_value

# Réorganiser les colonnes dans l'ordre demandé
column_order = [
    'champion', 'lane',
    f'patch_{PATCH_1}', f'winrate_{PATCH_1}', f'pickrate_{PATCH_1}', f'banrate_{PATCH_1}', f'games_{PATCH_1}', f'tier_{PATCH_1}',
    f'patch_{PATCH_2}', f'winrate_{PATCH_2}', f'pickrate_{PATCH_2}', f'banrate_{PATCH_2}', f'games_{PATCH_2}', f'tier_{PATCH_2}',
    'wr_change', 'pr_change', 'br_change',
    'fetch_timestamp'
]

comparison = comparison[column_order]

# Trier par plus gros changement de winrate
comparison_sorted = comparison.sort_values('wr_change', ascending=False)

print(f"✓ DataFrame de comparaison créé: {len(comparison)} champion-lane combinations\n")
print(f"📊 Structure du DataFrame:")
print(comparison_sorted.columns.tolist())
print(f"\n🔝 Top 10 buffs:")
display(comparison_sorted[['champion', 'lane', f'winrate_{PATCH_1}', f'winrate_{PATCH_2}', 'wr_change']].head(10))
print(f"\n🔻 Top 10 nerfs:")
display(comparison_sorted[['champion', 'lane', f'winrate_{PATCH_1}', f'winrate_{PATCH_2}', 'wr_change']].tail(10))

✓ DataFrame de comparaison créé: 783 champion-lane combinations

📊 Structure du DataFrame:
['champion', 'lane', 'patch_15.24', 'winrate_15.24', 'pickrate_15.24', 'banrate_15.24', 'games_15.24', 'tier_15.24', 'patch_16.1', 'winrate_16.1', 'pickrate_16.1', 'banrate_16.1', 'games_16.1', 'tier_16.1', 'wr_change', 'pr_change', 'br_change', 'fetch_timestamp']

🔝 Top 10 buffs:


,champion,lane,winrate_15.24,winrate_16.1,wr_change
92,caitlyn,jungle,4.35,100.0,95.65
723,xerath,jungle,11.11,100.0,88.89
676,veigar,jungle,16.67,100.0,83.33
344,lissandra,jungle,20.00,100.0,80.00
213,hwei,jungle,25.00,100.0,75.00
29,amumu,bottom,33.33,100.0,66.67
342,lillia,bottom,33.33,100.0,66.67
465,poppy,bottom,33.33,100.0,66.67
303,khazix,bottom,40.00,100.0,60.00
557,shyvana,bottom,41.67,100.0,58.33



🔻 Top 10 nerfs:


,champion,lane,winrate_15.24,winrate_16.1,wr_change
552,shen,bottom,36.54,16.67,-19.87
618,taric,middle,57.63,37.50,-20.13
516,rumble,bottom,59.52,37.50,-22.02
427,nidalee,bottom,44.55,22.22,-22.33
197,graves,support,40.48,14.29,-26.19
146,evelynn,top,39.72,12.50,-27.22
638,trundle,bottom,47.69,20.00,-27.69
187,gnar,support,42.11,14.29,-27.82
532,senna,middle,42.22,14.29,-27.93
99,camille,bottom,50.98,16.67,-34.31


## 💾 Sauvegarde des fichiers

In [12]:
# Sauvegarder en CSV
file_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Sauvegarder en CSV
df_patch_1.to_csv(f'patch_{PATCH_1}_{file_timestamp}.csv', index=False)
df_patch_2.to_csv(f'patch_{PATCH_2}_{file_timestamp}.csv', index=False)
comparison_sorted.to_csv(f'comparison_{PATCH_1}_vs_{PATCH_2}_{file_timestamp}.csv', index=False)

print(f"✓ Fichiers CSV sauvegardés:")
print(f"  - patch_{PATCH_1}_{file_timestamp}.csv ({len(df_patch_1)} lignes)")
print(f"  - patch_{PATCH_2}_{file_timestamp}.csv ({len(df_patch_2)} lignes)")
print(f"  - comparison_{PATCH_1}_vs_{PATCH_2}_{file_timestamp}.csv ({len(comparison_sorted)} lignes)")

print(f"\n📅 Données récupérées le: {fetch_timestamp}")

✓ Fichiers CSV sauvegardés:
  - patch_15.24_20260111_120415.csv (847 lignes)
  - patch_16.1_20260111_120415.csv (783 lignes)
  - comparison_15.24_vs_16.1_20260111_120415.csv (783 lignes)

📅 Données récupérées le: 2026-01-11 11:46:37
